In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
from google.colab.patches import cv2_imshow
import cv2
import os
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install keras==2.12
!pip install keras_applications --no-deps
!pip install split_folders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.12.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.applications import DenseNet201, ResNet152V2
from google.colab.patches import cv2_imshow
import cv2
import os
import pandas as pd
import splitfolders
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Resizing
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import seaborn as sn
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
drive.mount('/content/drive')

ruta_directorio = "/content/drive/MyDrive/UTEC/9_IA/2_Parcial_2/archive/data"
salida_ruta = "/content/drive/MyDrive/UTEC/9_IA/2_Parcial_2/archive/data/nueva"

splitfolders.ratio(ruta_directorio, output=salida_ruta, seed=42, ratio=(.6, .2, .2))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Copying files: 5631 files [37:59,  2.47 files/s]


In [ ]:
img_height,img_width=(224,224)
batch_size=32
train     = "/content/drive/MyDrive/UTEC/9_IA/2_Parcial_2/archive/data/nueva/train"
validatio = "/content/drive/MyDrive/UTEC/9_IA/2_Parcial_2/archive/data/nueva/test"
test      = "/content/drive/MyDrive/UTEC/9_IA/2_Parcial_2/archive/data/nueva/val"

In [ ]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 validation_split=0.4)
train_generator=train_datagen.flow_from_directory(
    train,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training")

valid_generator=train_datagen.flow_from_directory(
    validatio,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation")

test_generator=train_datagen.flow_from_directory(
    test,
    target_size=(img_height,img_width),
    batch_size=1,
    class_mode="categorical",
    subset="training")

Found 2027 images belonging to 5 classes.
Found 450 images belonging to 5 classes.
Found 676 images belonging to 5 classes.


In [ ]:
base_model = ResNet152V2(weights='imagenet', include_top = False)
base_model.trainable = False
model = Sequential()
model.add(Resizing(224,224))
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation ='relu'))
model.add(Dense(5, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
model.fit(train_generator,epochs=30)

234545216/234545216 [==============================] - 2s 0us/step
Epoch 1/30
64/64 [==============================] - 50s 573ms/step - loss: 1.2308 - accuracy: 0.9329
Epoch 2/30
64/64 [==============================] - 37s 576ms/step - loss: 0.3018 - accuracy: 0.9808
Epoch 3/30
64/64 [==============================] - 36s 560ms/step - loss: 0.3150 - accuracy: 0.9813
Epoch 4/30
64/64 [==============================] - 38s 591ms/step - loss: 0.3289 - accuracy: 0.9827
Epoch 5/30
64/64 [==============================] - 36s 563ms/step - loss: 0.0958 - accuracy: 0.9936
Epoch 6/30
64/64 [==============================] - 37s 582ms/step - loss: 0.0892 - accuracy: 0.9956
Epoch 7/30
64/64 [==============================] - 36s 554ms/step - loss: 0.0698 - accuracy: 0.9941
Epoch 8/30
64/64 [==============================] - 37s 573ms/step - loss: 0.0102 - accuracy: 0.9965
Epoch 9/30
64/64 [==============================] - 35s 549ms/step - loss: 0.1091 - accuracy: 0.9891
Epoch 10/30
64/64 [=====

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 224, 224, None)    0         
                                                                 
 resnet152v2 (Functional)    (None, None, None, 2048)  58331648  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 5)                 5125      
                                                                 
Total params: 161,098,245
Trainable params: 102,766,597
Non-trainable params: 58,331,648
_________________________________________________________________


In [ ]:
model.save("/content/drive/MyDrive/UTEC/9_IA/2_Parcial_2/archive/data/nueva/Modelo_ResNet152V2")

In [ ]:
base_model2 = DenseNet201(weights='imagenet', include_top = False)
base_model2.trainable = False
model2 = Sequential()
model2.add(Resizing(224,224))
model2.add(base_model2)
model2.add(Flatten())
model2.add(Dense(1024, activation ='relu'))
model2.add(Dense(5, activation = 'softmax'))
model2.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
model2.fit(train_generator,epochs=30)

74836368/74836368 [==============================] - 0s 0us/step
Epoch 1/30
64/64 [==============================] - 72s 717ms/step - loss: 1.8113 - accuracy: 0.9132
Epoch 2/30
64/64 [==============================] - 36s 557ms/step - loss: 0.5354 - accuracy: 0.9665
Epoch 3/30
64/64 [==============================] - 34s 528ms/step - loss: 0.3090 - accuracy: 0.9817
Epoch 4/30
64/64 [==============================] - 34s 525ms/step - loss: 0.1657 - accuracy: 0.9901
Epoch 5/30
64/64 [==============================] - 35s 552ms/step - loss: 0.2411 - accuracy: 0.9837
Epoch 6/30
64/64 [==============================] - 34s 528ms/step - loss: 0.1750 - accuracy: 0.9842
Epoch 7/30
64/64 [==============================] - 34s 523ms/step - loss: 0.1425 - accuracy: 0.9887
Epoch 8/30
64/64 [==============================] - 33s 522ms/step - loss: 0.2500 - accuracy: 0.9808
Epoch 9/30
64/64 [==============================] - 36s 554ms/step - loss: 0.1910 - accuracy: 0.9877
Epoch 10/30
64/64 [=======

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 224, 224, None)    0         
                                                                 
 resnet152v2 (Functional)    (None, None, None, 2048)  58331648  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 5)                 5125      
                                                                 
Total params: 161,098,245
Trainable params: 102,766,597
Non-trainable params: 58,331,648
_________________________________________________________________


In [ ]:
model.save("/content/drive/MyDrive/UTEC/9_IA/2_Parcial_2/archive/data/nueva/Modelo_DenseNet201")